# Concurrencia y Paralelismo Avanzado

Cuando múltiples hilos o procesos necesitan acceder a los mismos datos, surgen nuevos desafíos: ¿cómo nos aseguramos de que no se "pisen" el trabajo y corrompan la información? ¿Cómo pueden comunicarse entre sí?

En este notebook, exploraremos las herramientas que Python nos ofrece para gestionar la **sincronización** y el **intercambio de datos** de forma segura.

## 1. Sincronización de Hilos con `Lock`

**El Problema (Condiciones de Carrera):** Imagina dos cajeros (hilos) intentando depositar dinero en la misma cuenta (una variable compartida) al mismo tiempo. Ambos leen el saldo, le suman el depósito y lo guardan. Si sus acciones se entrelazan, uno de los depósitos se puede perder.

**La Solución (`threading.Lock`):** Un `Lock` (o "cerrojo") es como la **llave de un baño público**. Solo una persona (un hilo) puede tener la llave y entrar a la vez. El resto debe esperar afuera hasta que la llave esté libre. En código, usamos el bloque `with lock:` para asegurarnos de que solo un hilo acceda a la variable `saldo` a la vez.

In [1]:
import threading

# Variable compartida que ambos hilos intentarán modificar
saldo = 0
# Creamos el "cerrojo" para proteger el acceso a la variable 'saldo'
lock = threading.Lock()

def depositar(dinero):
    global saldo
    # Simulamos 100,000 pequeños depósitos
    for _ in range(100000):
        # 'with lock:' adquiere el cerrojo antes de entrar al bloque
        # y lo libera automáticamente al salir.
        with lock:
            # Esta operación (lectura-modificación-escritura) ahora es "atómica".
            # Ningún otro hilo puede interrumpirla.
            saldo += dinero

# Creamos dos hilos que ejecutarán la misma función de depositar
hilos = []
for _ in range(2):
    hilo = threading.Thread(target=depositar, args=(1,))
    hilos.append(hilo)
    hilo.start() # Iniciamos la ejecución de los hilos

# Esperamos a que ambos hilos terminen su trabajo
for hilo in hilos:
    hilo.join()

# Gracias al Lock, el resultado es el esperado (200,000)
print(f"Saldo final: {saldo}")

Saldo final: 200000


## 2. Compartir Datos entre Procesos

A diferencia de los hilos, los procesos no comparten memoria. Son como dos programas completamente aislados. Para que puedan comunicarse, necesitamos canales explícitos.

### `multiprocessing.Queue`
**La Analogía:** Es como un **tubo neumático** entre dos oficinas (procesos). Una oficina puede meter un mensaje (`.put()`) en el tubo, y la otra oficina puede sacarlo (`.get()`) al otro lado. Es una forma segura de pasar datos en una dirección.

In [2]:
import multiprocessing

# Esta función se ejecutará en un proceso separado
def calcular_cuadrado(numeros, cola):
    """Calcula el cuadrado de cada número y pone el resultado en la cola."""
    for n in numeros:
        cola.put(n * n)

if __name__ == "__main__":
    numeros = [1, 2, 3, 4, 5]
    # Creamos la 'cola' o canal de comunicación
    cola = multiprocessing.Queue()

    # Creamos el nuevo proceso, pasándole la cola como argumento
    proceso = multiprocessing.Process(target=calcular_cuadrado, args=(numeros, cola))

    proceso.start() # Iniciamos el proceso
    proceso.join()  # Esperamos a que termine

    # En el proceso principal, vaciamos la cola para obtener los resultados
    print("Resultados recibidos desde el otro proceso:")
    while not cola.empty():
        print(cola.get())

Resultados recibidos desde el otro proceso:


### `multiprocessing.Manager`
**La Analogía:** Es como crear una **pizarra compartida** en la nube. Varios procesos pueden ver y escribir en la misma lista o diccionario (`manager.list()`, `manager.dict()`) al mismo tiempo, y el `Manager` se encarga de que los cambios se sincronicen correctamente.

In [3]:
import multiprocessing

# Función que cada proceso ejecutará
def agregar_valores(lista_compartida, id_proceso):
    """Añade números a la lista compartida."""
    for i in range(5):
        valor = id_proceso * 10 + i
        lista_compartida.append(valor)
        print(f"Proceso {id_proceso} añadió: {valor}")

if __name__ == "__main__":
    # Creamos un Manager que gestionará los objetos compartidos
    with multiprocessing.Manager() as manager:
        # Pedimos al manager que nos cree una lista compartida
        lista_compartida = manager.list()

        # Creamos dos procesos que trabajarán sobre la MISMA lista
        proceso1 = multiprocessing.Process(target=agregar_valores, args=(lista_compartida, 1))
        proceso2 = multiprocessing.Process(target=agregar_valores, args=(lista_compartida, 2))

        proceso1.start()
        proceso2.start()

        proceso1.join()
        proceso2.join()

        # El resultado final contiene los valores de ambos procesos
        print(f"\nLista compartida final: {sorted(list(lista_compartida))}")


Lista compartida final: []
